# Enable CloudWatch Logging for Amazon Bedrock

In [1]:
import boto3
import json
import os

bedrock = boto3.client('bedrock', region_name="us-west-2")

In [2]:
from helpers.CloudWatchHelper import CloudWatch_Helper
cloudwatch = CloudWatch_Helper()

In [4]:
log_group_name = '/my/amazon/bedrock/logs'

In [5]:
cloudwatch.create_log_group(log_group_name)

Log group '/my/amazon/bedrock/logs' created successfully.


In [6]:
loggingConfig = {
    'cloudWatchConfig': {
        'logGroupName': log_group_name,
        'roleArn': os.environ['LOGGINGROLEARN'],
        'largeDataDeliveryS3Config': {
            'bucketName': os.environ['LOGGINGBUCKETNAME'],
            'keyPrefix': 'amazon_bedrock_large_data_delivery',
        }
    },
    's3Config': {
        'bucketName': os.environ['LOGGINGBUCKETNAME'],
        'keyPrefix': 'amazon_bedrock_logs',
    },
    'textDataDeliveryEnabled': True,
}

In [7]:
bedrock.put_model_invocation_logging_configuration(loggingConfig=loggingConfig)

{'ResponseMetadata': {'RequestId': '25c8d961-188f-47ae-941c-a94eeb2f2d27',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 19 Feb 2026 12:27:08 GMT',
   'content-type': 'application/json',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '25c8d961-188f-47ae-941c-a94eeb2f2d27'},
  'RetryAttempts': 0}}

In [8]:
bedrock.get_model_invocation_logging_configuration()

{'ResponseMetadata': {'RequestId': '9c6e8797-3e84-4cb6-8306-99a42b9f8ce8',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 19 Feb 2026 12:27:08 GMT',
   'content-type': 'application/json',
   'content-length': '636',
   'connection': 'keep-alive',
   'x-amzn-requestid': '9c6e8797-3e84-4cb6-8306-99a42b9f8ce8'},
  'RetryAttempts': 0},
 'loggingConfig': {'cloudWatchConfig': {'logGroupName': '/my/amazon/bedrock/logs',
   'roleArn': 'arn:aws:iam::766939550768:role/c99355a2566044l13869611t1w7669395507-LoggingIAMRole-rlUUl2Gt4wfV',
   'largeDataDeliveryS3Config': {'bucketName': 'c99355a2566044l13869611t1w76693955-loggings3bucket-gyxg8rsy6oit',
    'keyPrefix': 'amazon_bedrock_large_data_delivery'}},
  's3Config': {'bucketName': 'c99355a2566044l13869611t1w76693955-loggings3bucket-gyxg8rsy6oit',
   'keyPrefix': 'amazon_bedrock_logs'},
  'textDataDeliveryEnabled': True,
  'imageDataDeliveryEnabled': True,
  'embeddingDataDeliveryEnabled': True}}

In [9]:
bedrock_runtime = boto3.client('bedrock-runtime', region_name="us-west-2")

In [10]:
prompt = "Write an article about the fictional planet Foobar."

kwargs = {
    # model updated
    "modelId": "us.amazon.nova-lite-v1:0",
    "contentType": "application/json",
    "accept": "*/*",
    "body": json.dumps(
        {   # updated due to the use of a newer model
            "messages": [{"role": "user", "content": [{"text": prompt}]}],
            "inferenceConfig": {
                "maxTokens": 512,
                "temperature": 0.7,
                "topP": 0.9
            }
        }
    )
}

response = bedrock_runtime.invoke_model(**kwargs)
response_body = json.loads(response.get('body').read())

content_list = response_body["output"]["message"]["content"]
text_block = next((item for item in content_list if "text" in item), None)
generation = text_block["text"] if text_block else ""

print(generation)

# Discovering the Enigmatic Planet Foobar

## Introduction
Nestled in the uncharted reaches of the Andromeda Galaxy lies a celestial wonder that has captivated the imaginations of scientists, explorers, and science fiction enthusiasts alike: the fictional planet Foobar. With its striking landscapes, unique ecosystems, and intriguing inhabitants, Foobar stands as a beacon of mystery and allure in the vast expanse of the cosmos.

## A Celestial Marvel

### Geographical Features
Foobar is a planet of contrasts and wonders. Its surface is a mosaic of vibrant colors, with sprawling forests of bioluminescent flora, towering crystalline mountains, and vast oceans of luminescent liquid. The planet's atmosphere shimmers with hues of violet and teal, creating a breathtaking spectacle that changes with the shifting seasons.

#### The Crystalline Peaks
One of Foobar's most remarkable features is the Crystalline Peaks, a range of mountains that sparkle with a spectrum of colors. These peaks are not

In [11]:
cloudwatch.print_recent_logs(log_group_name)

Permissions are correctly set for Amazon Bedrock logs.
-------------------------

